# Cosine Similarity Substitution
`w266 Final Project: Crosslingual Word Embeddings`

Instead of traning on randomly substituted words, here we'll choose the translation that is closest to the context embedding vector.

# Notebook Setup

In [1]:
# general imports
from __future__ import print_function
import time
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

# tell matplotlib not to open a new window
%matplotlib inline

# autoreload modules
%load_ext autoreload
%autoreload 2

Set base paths depending on your machine.

In [2]:
BASE = '/home/mmillervedam/Data'
PROJ = '/home/mmillervedam/ProjectRepo'
#PROJ = '/Users/mona/OneDrive/repos/final_proj/W266-Fall-2017-Final-Project'

# raw data
FULL_EN_ES = "/home/miwamoto/en_es_shuf.txt"
FULL_EN_IT = "/home/miwamoto/en_it_shuf.txt"

# vocabularies
VOCAB_EN_ES = BASE + '/vocab/en_es_index.pkl'
VOCAB_EN_IT = BASE + '/vocab/en_it_index.pkl'

# panlex dicts
PANLEX_EN_ES = BASE + '/panlex/en_es_dict.pkl'
PANLEX_EN_IT = BASE + '/panlex/en_it_dict.pkl'

# directory to save pickled embeddings
SAVE_TO = BASE + '/embeddings'

# English - Spanish
### Load Data, Dict & Vocab

In [46]:
from parsing import Corpus, BilingualVocabulary, batch_generator

In [47]:
# load corpus
en_es_data = Corpus(FULL_EN_ES)

In [48]:
# load panlex dictionary
with open(PANLEX_EN_ES,'rb') as f:
    en_es_translations = pickle.load(f)

In [49]:
# load vocabulary
en_es_vocab = BilingualVocabulary([], languages = ('en','es'))
with open(VOCAB_EN_ES,'rb') as f:
    en_es_vocab.load_from_index(pickle.load(f))

In [50]:
# confirmations
print('... loaded %s panlex translations'%(len(en_es_translations)))
print('... loaded %s word %svocabulary'%(en_es_vocab.size,en_es_vocab.language))

... loaded 702982 panlex translations
... loaded 200003 word ('en', 'es')vocabulary


### Initialize the model

In [54]:
from models import BiW2V_nn

EMBEDDING_SIZE = 200

# create model
model = BiW2V_nn(bilingual_dict = en_es_translations,
                 vocab = en_es_vocab, H = EMBEDDING_SIZE)

# intialize TF graphs
model.BuildCoreGraph()
model.BuildTrainingGraph()
model.BuildValidationGraph()

... TF graph created for BiW2V model.
... TF graph created for BiW2V training.
... TF graph created for BiW2V validation.


### Training

In [55]:
# training parameters
nBATCHES = 6000 # less than 1 epoch
BATCH_SIZE = 48
WINDOW_SIZE = 4
MAX_EPOCHS = 5 # fail safe
DATA_GENERATOR = batch_generator(en_es_data, en_es_vocab, BATCH_SIZE, WINDOW_SIZE, MAX_EPOCHS)
TEST_WORDS = en_es_vocab.to_ids(['en_the','en_last', 'es_mundo', 'es_real'])

In [56]:
# train
start = time.time()
model.train(nBATCHES, DATA_GENERATOR, TEST_WORDS, learning_rate = 0.5)
tot = (time.time() - start)
print('... {} batches trained in {} seconds'.format(nBATCHES, tot))

... Model Initialized
	 <tf.Variable 'Embedding_Layer/ContextEmbeddings:0' shape=(200003, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/WordEmbeddings:0' shape=(200003, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/b:0' shape=(200003,) dtype=float32_ref>
... Starting Training
... STEP 0 : Average Loss : 0.0226768477758
   [en_the] closest:  en_vestry, en_villager, es_kraus, es_casarla, es_gangrena, es_monofisita, es_valaco, en_doubleheader,
   [en_last] closest:  es_djokovic, en_0-14, en_katakana, es_sentimiento, en_conceptualized, es_pretendidamente, en_scar, en_8,500,
   [es_mundo] closest:  en_sequestered, es_castrillo, en_hitoshi, en_woodroffe, es_,...), en_deeply, es_reivindicaciones, en_behring,
   [es_real] closest:  en_isidor, en_1010, es_barbudo, en_pass, es_bandurria, en_thermopylae, en_domains, en_interpreted,
... STEP 600 : Average Loss : 7.83098859429
... STEP 1200 : Average Loss : 6.52981234233
   [en_the] closest:  <s>, es_de, en_,, es_gangrena, es_,, en_ve

### `Bu Hao` (a.k.a. `Bad News Bears` a.k.a `Yoku Nai`)
```
... 1000 batches trained in 553.925096989 seconds
... 6000 batches trained in 5380.98421979 seconds
```

### Save the Embeddings such as they are.

In [60]:
# context 
filename = SAVE_TO + '/en_es_nn_6K_cw4_V_dec19.pkl'
with open(filename, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model.context_embeddings, f, pickle.HIGHEST_PROTOCOL)

# word
filename = SAVE_TO + '/en_es_nn_600K_cw4_U_dec19.pkl'
with open(filename, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model.word_embeddings, f, pickle.HIGHEST_PROTOCOL)